In [39]:
import numpy as np
import pandas as pd
import os
import shutil
from checkRF_Score import *

In [40]:
dataset_path = join_dir("dataset", "asteroid_dataset", "withils")

In [41]:
# in asteroid dataset
gt_file_name="true.true.geneTree.newick"
# species_tree_name="speciesTree.newick"
species_tree_name="fastrfs-raxml-ng_greedy.GTR+G.speciesTree.newick"

In [42]:
# for our experiment
gt_output_folder= "output"
gt_output_file_name="1_gt.tre"
true_species_tree_name= "true-species.out.tree"

In [43]:
!rm -rf $gt_output_folder
!mkdir -p $gt_output_folder

In [44]:
import re

def remove_one(input_str):
    output_str = re.sub(r"1(?=[^\d_])", "", input_str)
    return output_str

def remove_numbers_after_colon(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    output_str = remove_one(output_str)
    return output_str

def remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    return output_str

In [45]:
# provide families folder path
def get_all_GTs(families_folder):
    all_gts = []
    for dir in os.listdir(families_folder):
        # print(dir)
        # save mapping from mapping folder
        mapping={}
        mapping_dir="mappings"
        mapping_file_path=families_folder+"/"+dir+"/"+mapping_dir+"/treerecs_mapping.link"
        if os.path.exists(mapping_file_path):
            # print(mapping_file_path)
            with open(mapping_file_path) as f:
                for line in f:
                    # print(line)
                    (key, val) = line.split()
                    mapping[key] = val
        else:
            print("mapping file not found")
        if len(mapping) == 0:
            print("mapping is empty")
        # print(mapping)
        gt_tree_dir="gene_trees"
        gt_tree_file_path=families_folder+"/"+dir+"/"+gt_tree_dir+"/"+gt_file_name
        if not os.path.exists(gt_tree_file_path):
            print("file not exists")
            exit(1)
        with open(gt_tree_file_path) as f:
            line = f.readline()
            line = remove_numbers_after_colon(line)
            # print(line)
            for key in mapping:
                line = line.replace(key, mapping[key])
            all_gts.append(line)
            # print(line)
    #output to a file in output folder
    with open(gt_output_folder+"/"+gt_output_file_name, "w") as f:
        for gt in all_gts:
            f.write(gt+"\n")

In [46]:
def get_True_Species_Tree(source_folder):
    # copy species tree to output folder
    # species_tree_path = source_folder+"/species_trees/"+species_tree_name
    species_tree_path = join_dir(source_folder, "species_trees", species_tree_name)
    with open(species_tree_path) as f:
        line = f.readline()
        line = remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(line)
        with open(gt_output_folder+"/"+true_species_tree_name, "w") as f:
            f.write(line)


In [47]:
def getTaxaSet(t):
    return t.replace('(', '').replace(')', '').split(';')[0].split(':')[0].split(',')

In [48]:
def get_INPUT_DATA():
    OUTGROUP = 'GAL'
    st = open('output/input.nwk')
    st = st.read()
    gts = open('output/1_gt.tre')

    gts = gts.read().split(';\n')[:-1] 

    # print(len(gts))

    # print(st)
    taxaSet = getTaxaSet(st)
    # print(taxaSet)
    matrix = np.zeros((len(taxaSet), len(gts)), dtype=int)


    # print(len(gts))
    for gt_idx, gt in  enumerate(gts) :
        # print(gt)
        cur_taxaSet = getTaxaSet(gt)
        # print(cur_taxaSet)
        for i, tx in enumerate(taxaSet):
            if tx  in cur_taxaSet or tx == OUTGROUP:
                matrix[i][gt_idx] = 1
    with open('output/input.data', 'w', newline='') as f:
        print(matrix.shape[0], matrix.shape[1], file=f)
        for i in range(len(taxaSet)):
            # f.write(matrix[i].tostring(), taxaSet[i])
            print(*matrix[i],taxaSet[i], end='\n', file=f)

In [49]:
def getTerrace():
    !rm -rf terrace_output
    !mkdir -p terrace_output
    !./bin/raxml-ng output/input.nwk output/input.data > terrace_output/terrace_output.txt

In [50]:
def getConsensusTree():
    # get consensus tree
    terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
    consensus_dir = join_dir(terrace_dir, "consensus")
    !rm -rf "$consensus_dir"
    !mkdir "$consensus_dir"

    if not dir_exists_abs(terrace_dir):
        print("Directory does not exist: {}".format(terrace_dir))
        exit(1)
    for terrace_file in os.listdir(terrace_dir):
        file_path = join_dir(terrace_dir, terrace_file)
        if file_path.endswith(".txt"):
            print("Processing file: {}".format(file_path))
            terrace_file_name = terrace_file.split(".")[0]
            !./bin/raxml-ng --consense MRE --tree "$file_path" --prefix "$consensus_dir"/"$terrace_file_name"

In [51]:
def get_ASTRAL_tree():
    !echo "ASTRAL Tree generating : sayem"
    !rm -rf in/
    !rm -rf out/
    !mkdir in
    !cp output/1_gt.tre in/
    !python3 postprocessor.py
    !cp out/complete/astral/1_gt_species.out.tre output/input.nwk

In [52]:
def get_ASTRAL_RF_Score(true_tree):
    astral_rf_score = calculate_RF_distance_for_a_ST(join_dir(gt_output_folder, "input.nwk"), true_tree)
    print("astral_rf_score --> ", astral_rf_score)
    return astral_rf_score

In [53]:
def get_Terrace_RF_Scores(true_tree):
    
    #get RF score for each tree in Terrace
    terrace_rf_scores=calculate_RF_distance_for_terrace(join_dir(TERRACE_OUTPUT_FOLDER, "terrace_output.txt"), true_tree)
    # print("terrace_rf_scores", terrace_rf_scores)
    if terrace_rf_scores is not None:
        return terrace_rf_scores
    else:
        return []

In [54]:
def get_Consensus_RF_score(true_tree):
    terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
    consensus_dir = join_dir(terrace_dir, "consensus")
    CONSENSUS_TREE_EXTENSION = "consensusTreeMRE"
    file_path = join_dir(consensus_dir, "terrace_output.raxml.consensusTreeMRE")
    print("Processing file: {}".format(file_path))
    print("True Tree --> ", true_tree)
    # remove_numbers_from_consensus_tree(file_path)
    if file_exists_absolute(file_path):
        dist = calculate_RF_distance_for_a_ST(file_path, true_tree)
        print("Consensus RF distance: {}".format(dist))
        return dist
    else:
        print("File does not exist: {}".format(file_path))
        return -1

In [55]:
def Keep_few_STs_in_Terrace(KEEPING_STs_NUM):
    num_of_trees_in_terrace = 0
    for file in os.listdir("terrace_output"):
        if file.endswith(".txt"):
            file_path = "terrace_output/"+file
            df = pd.read_csv(file_path)
            num_of_trees_in_terrace = len(df)
            if len(df) > KEEPING_STs_NUM:
                df = df.iloc[:KEEPING_STs_NUM]
                df.to_csv(file_path, index=False)
    return num_of_trees_in_terrace

In [56]:
def get_stats_of_terrace(terrace_rf_scores,true_tree, astral_rf):
    return calculate_RF_AND_generate_stats_OF_TERRACE(terrace_rf_scores,true_tree, astral_rf)

In [57]:
csv_out = []
csv_out.append(["Model Condition", "RF Score","Consensus RF" ,"Trees Better than ASTRAL", "Trees Equal to ASTRAL", "Tress worse than ASTRAL"])
i=0
for model_condition in sorted(os.listdir(dataset_path)):
    # if i==3:
        # i+=1
        # continue
    # print(model_condition)
    if "s125" not in model_condition:
        continue
    
    print("Model Condition --> ", model_condition)

    source_folder = join_dir(dataset_path, model_condition)
    # print(source_folder)
    # source_folder="dataset/asteroid_dataset/withils/ssim_veryhighmiss_s25_f1000_sites100_GTR_bl1.0_d0.0_l0.0_t0.0_gc0.0_p0.0_pop50000000_ms0.6_mf0.6_seed3000"
    families_folder = join_dir(source_folder, "families")
    get_all_GTs(families_folder)
    get_True_Species_Tree(source_folder)
    
    true_tree = join_dir(gt_output_folder, true_species_tree_name)
    
    get_ASTRAL_tree()

    get_INPUT_DATA()

    getTerrace()

    MAX_STs_IN_TERRACE = 20000 
    number_of_trees_in_terrace = Keep_few_STs_in_Terrace(MAX_STs_IN_TERRACE)
    #get rf scores of each tree in terrace
    terrace_file_path = join_dir(TERRACE_OUTPUT_FOLDER, "terrace_output.txt")
    # terrace_rf_scores = calculate_RF_distance_for_terrace(terrace_file_path, true_tree)

    #consensus of trees of terrace
    getConsensusTree()
    
    # rf scores
    astral_rf = get_ASTRAL_RF_Score(true_tree)
    if number_of_trees_in_terrace >1:
        consensus_rf = get_Consensus_RF_score(true_tree)

        # get stats of how many trees are better/eq/worse than ASTRAL
        better, equal, worse = get_stats_of_terrace(terrace_file_path,true_tree, astral_rf)

        # csv_out.append([model_condition, astral_rf, consensus_rf])
        csv_out.append([model_condition, astral_rf, consensus_rf, better, equal, worse]) 
    
    i+=1
    if i==20:
        break

stat_fl = 'stats/Summary.csv'
with open(stat_fl, 'w') as stat_file:
    csv.writer(stat_file).writerows(csv_out)

# stat_fl = 'stats/Summary_terrace.csv'
# with open(stat_fl, 'w') as stat_file:
#     csv.writer(stat_file).writerows(csv_out_terrace) 

Model Condition -->  ssim_veryhighmiss_s50_f1000_sites500_GTR_bl1.0_d0.0_l0.0_t0.0_gc0.0_p0.0_pop50000000_ms0.6_mf0.6_seed3000
ASTRAL Tree generating : sayem

================== ASTRAL ===================== 

This is ASTRAL version 5.6.3
Gene trees are treated as unrooted
840 trees read from /Users/sayemhasan/Desktop/Thesis/Terraces/raxml-ng/in/1_gt.tre
All output trees will be *arbitrarily* rooted at 19

======== Running the main analysis
Number of taxa: 50 (50 species)
Taxa: [19, 17, 25, 15, 36, 31, 33, 26, 42, 1, 27, 20, 48, 28, 13, 5, 9, 50, 35, 39, 46, 43, 18, 4, 3, 11, 8, 7, 6, 34, 30, 10, 38, 45, 49, 41, 22, 23, 14, 24, 12, 32, 47, 29, 40, 21, 2, 37, 44, 16]
Taxon occupancy: {44=73, 45=90, 46=226, 47=89, 48=146, 49=86, 50=179, 10=152, 11=123, 12=152, 13=342, 14=164, 15=264, 16=88, 17=146, 18=126, 19=167, 1=267, 2=114, 3=235, 4=248, 5=182, 6=159, 7=224, 8=177, 9=217, 20=49, 21=94, 22=273, 23=132, 24=134, 25=182, 26=50, 27=236, 28=241, 29=112, 30=75, 31=157, 32=184, 33=259, 34=194